In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
mnist = input_data.read_data_sets(r"C:\Users\zdwxx\Downloads\Compressed\MNIST_data"
                                 , one_hot=True)

lr = 0.001
training_iters = 100000
batch_size = 128

n_inputs = 28 # MNIST输入的数据
n_steps = 28
n_hidden_units = 128 # 隐藏层的数量
n_classes = 10 # 0~9

x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

# define weights
weights = {
    # (28, 128)
    "in" : tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    "out" : tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}

# define biases
biases = {
    # (128, )
    "in" : tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ])), 
    # (10, )
    "out" : tf.Variable(tf.constant(0.1, shape=[n_classes, ]))
}

def RNN(X, weights, biases):
    
    # hidden layer for input to cell
    # X(128batch, 28 steps, 28 inputs)
    # x--> (128*28, 28inputs)
    X = tf.reshape(X, (-1, n_inputs))
    X_in = tf.matmul(X, weights["in"]) + biases["in"]
    # X_in -> (128batch, 28steps, 128hidden)
    X_in = tf.reshape(X_in, (-1, n_steps, n_hidden_units))
    
    # cell
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units, forget_bias=1.0, 
                                            state_is_tuple=True)
    # 两个state，c_state主线， m_state分线
    _init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
    # 计算结果
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, X_in, 
                                       initial_state=_init_state, 
                                      time_major=False)
    
    # hidden layer for outputs as final results
    results = tf.matmul(states[1], weights["out"]) + biases["out"]

    return results

tf.reset_default_graph()
prediction = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=prediction, lables=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuary = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict={x : batch_xs, y:batch_ys})
        
        if step % 20 == 0:
            print(sess.run(accuary, feed_dict={x : batch_xs, y : batch_ys}))
        step += 1


Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\train-images-idx3-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\train-labels-idx1-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\t10k-labels-idx1-ubyte.gz


ValueError: Tensor("rnn/Const:0", shape=(1,), dtype=int32) must be from the same graph as Tensor("ExpandDims:0", shape=(1,), dtype=int32).